In [59]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.datasets import load_sample_image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.ndimage import shift
import matplotlib.pyplot as plt

In [60]:
aug = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.2,  
    height_shift_range=0.2,  
    brightness_range=[0.7, 1.3],  
    fill_mode="constant",  
    cval=0  
)

In [61]:

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)



In [62]:
augmented_images = []
augmented_labels = []
for image, label in zip(x_train, y_train):
    image = np.expand_dims(image, axis=0)  
    augment_iter = aug.flow(image, batch_size=1) 
    for _ in range(2):  
        aug_image = next(augment_iter)[0]
        augmented_images.append(aug_image)
        augmented_labels.append(label)


augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)
x_train_augmented = np.concatenate([x_train, augmented_images])
y_train_augmented = np.concatenate([y_train, augmented_labels])

print(x_train.shape)
print(x_train_augmented.shape)

(60000, 28, 28, 1)
(180000, 28, 28, 1)


In [63]:
x_train, x_val, y_train, y_val = train_test_split(x_train_augmented, y_train_augmented, test_size=0.2, random_state=60)

In [64]:
x_train_flat = x_train.reshape(-1, 28 * 28)
x_val_flat = x_val.reshape(-1, 28 * 28)
x_test_flat = x_test.reshape(-1, 28 * 28)


In [65]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_flat)
x_val_scaled = scaler.transform(x_val_flat)
x_test_scaled = scaler.transform(x_test_flat)

In [66]:
x_train = x_train_flat.reshape(-1, 28, 28, 1)
x_val = x_val_flat.reshape(-1, 28, 28, 1)
x_test = x_test_flat.reshape(-1, 28, 28, 1)

In [67]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, 5, padding="same", input_shape=[28, 28, 1]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPooling2D(2),
    
    tf.keras.layers.Conv2D(128, 3, padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Conv2D(128, 3, padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPooling2D(2),
    
    tf.keras.layers.Conv2D(256, 3, padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Conv2D(256, 3, padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPooling2D(2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(64),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(10, activation="softmax")
])


C:\Users\marce\anaconda4\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [68]:
print(model.summary)

<bound method Model.summary of <Sequential name=sequential_6, built=True>>


In [69]:
def lr_schedule(epoch):
    if epoch < 10:
        return 0.01
    elif epoch < 20:
        return 0.05
    elif epoch < 30:
        return 0.001
    elif epoch < 40:
        return 0.0005
    else:
        return 0.0001


In [70]:
callbacks = [
    tf.keras.callbacks.LearningRateScheduler(lr_schedule, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_accuracy")
]

In [71]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.99, nesterov=True)
model.compile(
    optimizer=optimizer,
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=callbacks
)


Epoch 1: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/50
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 0s 419ms/step - accuracy: 0.1294 - loss: 2.2728

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")